# ABBA cell count analysis

This notebook is the last step in the ABBA whole-brain cell counting analysis.  
It assumes you have done the following steps:
- Alignment of brain slices in ABBA, exported to a QuPath project.
- Detected cells of interest in QuPath. The detections should be exported to ```.csv``` files (one per slice) in a folder called ```results```. 
- If there are regions to exclude, you should have drawn them and exported to ```.txt``` files (one per slice) in a folder called ```regions_to_exclude```.

Run this notebook to load the cell counts and do analysis on them. 

## Before we start ...
### Set parameters

In [ ]:
EXPERIMENT_DIRECTORY = "p4"
USE_LOCAL_DATA = False                                      # if False, it tries to read the data on the laboratory's server
IS_COLLABORATION_PROJ = False
import os
COLLABORATION_DIRECTORY = os.path.join("Mathias Schmidt", "soumnya")

# ###################################### LOCAL DIRECTORIES ######################################
# DATA_ROOT  = f"../data/experiments/soumnya/{EXPERIMENT_DIRECTORY}"
# PLOTS_ROOT = f"../plots/soumnya/{EXPERIMENT_DIRECTORY}/"
DATA_ROOT  = f"../data/experiments/{EXPERIMENT_DIRECTORY}"
PLOTS_ROOT = f"../plots/{EXPERIMENT_DIRECTORY}"

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
BRANCHES_TO_EXCLUDE = ["retina", "VS", "grv", "fiber tracts", "CB"]
USE_LITERATURE_REUNIENS = True                             # add a 'REtot' region, merging the following regions: 'PR', 'RE', 'Xi', 'RH'
PLOT_ALLENBRAIN_HIERARCHY = False
PLOT_ANIMALS_ROOTS = True
PLOT_COEFFICIENT_OF_VARIATION = True
PLOT_COEFFICIENT_OF_VARIATION_THRESHOLD = 1

# ###################################### DATA LOAD OPTIONS ######################################
AREA_KEY = "Area um^2"
TRACER_KEY = "Num AF647"
MARKER = "CFos"

# ###################################### DATA SAVE OPTIONS ######################################
ANIMAL_BRAIN_MODE = "sum"                                   # available options are: 'sum', 'mean'/'avg', 'std', 'variation'/'cvar'
SAVE_ANIMALS = True
SAVE_GROUPS = True

# ###############################################################################################
from collections import namedtuple
GroupDirectory = namedtuple("GroupDirectory", "name dirs")

In [ ]:
# SOUMNYA FEMALES+MALES - 2 Groups {Stress|Control}
groups = [
    GroupDirectory(
        name="Control",
        dirs=["41CM", "42CF", "43CM", "44CF", "49CM", "50CF", "51CM", "52CF", "58CF", "60CF", "74CF", "76CF", "81CM", "83CM", "89CM", "91CM"]
    ),
    GroupDirectory(
        name="Stress",
        dirs=["45SM", "46SF", "47SM", "48SF", "53SM", "54SF", "55SM", "56SF", "62SF", "64SF", "78SF", "80SF", "85SM", "87SM", "93SM", "95SM"]
    ),
]

In [ ]:
# SOUMNYA ALL - 2 Groups {Stress|Control} + 2 Groups {Males|Females}
groups = [
    GroupDirectory(
        name="Control (Females)",
        dirs=["42CF", "44CF", "50CF", "52CF", "58CF", "60CF", "74CF", "76CF"]
    ),
    GroupDirectory(
        name="Stress (Females)",
        dirs=["46SF", "48SF", "54SF", "56SF", "62SF", "64SF", "78SF", "80SF"]
    ),
    GroupDirectory(
        name="Control (Males)",
        dirs=["41CM", "43CM", "49CM", "51CM", "81CM", "83CM", "89CM", "91CM"]
    ),
    GroupDirectory(
        name="Stress (Males)",
        dirs=["45SM", "47SM", "53SM", "55SM", "85SM", "87SM", "93SM", "95SM"]
    ),
]

In [ ]:
# SHILA - 3 Groups {Control|Stress|Resilient}
groups = [
    GroupDirectory(
        name="Control",
        dirs=["16C", "17C", "19C"] # 18C
    ),
    GroupDirectory(
        name="Stress",
        dirs=["5S", "8S", "10S", "13S", "14S"]
    ),
    GroupDirectory(
        name="Resilient",
        dirs=["1R", "2R", "3R", "4R", "11R"]
    ),
]

In [ ]:
# SHILA - 2 Groups {Control|Stress+Resilient}
groups = [
    GroupDirectory(
        name="Control",
        dirs=["16C", "17C", "19C"]
    ),
    GroupDirectory(
        name="Stress",
        dirs=["5S", "10S", "13S", "14S", "1R", "2R", "3R", "4R", "11R"]
    ),
]

In [ ]:
# TEST DATA - Sample data
groups = [
    GroupDirectory(
        name="Control",
        dirs=["Control_16C", "Control_17C", "Control_18C", "Control_19C", "Control_42C"]
    ),
    GroupDirectory(
        name="Stress",
        dirs=["Stress_5S", "Stress_8S", "Stress_10S", "Stress_13S"]
    ),
    GroupDirectory(
        name="Resilient",
        dirs=["Resilient_1R", "Resilient_2R", "Resilient_3R", "Resilient_4R", "Resilient_11R"]
    ),
]

In [ ]:
# P4 DATA - Sample data
groups = [
    GroupDirectory(
        name="SNi",
        dirs=["206", "208", "210", "211"]
    ),
    GroupDirectory(
        name="Sham",
        dirs=["212", "213", "214", "220"]
    ),
]

# Script's code

In [ ]:
import os
import sys
from typing import List

project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

In [ ]:
if not USE_LOCAL_DATA:
    match sys.platform:
        case "darwin":
            mnt_point = "/Volumes/Ricerca/"
            
        case "linux":
            mnt_point = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/"
        case "win32":
            mnt_point = "\\\\ich.techosp.it\\Ricerca\\"
        case _:
            raise Exception(f"Can't find the 'Ricerca' folder in the server for '{sys.platform}' operative system. Please report the developer (Carlo)!")
    if not os.path.isdir(mnt_point):
        raise Exception(f"Could not read '{mnt_point}'. Please be sure you are connected to the server.")
    if IS_COLLABORATION_PROJ:
        DATA_ROOT  =  os.path.join(mnt_point, "Lab Matteoli", "Silva", "collaborations", COLLABORATION_DIRECTORY, "data", EXPERIMENT_DIRECTORY)
        PLOTS_ROOT = os.path.join(mnt_point, "Lab Matteoli", "Silva", "collaborations", "Mathias Schmidt", "soumnya", "results", EXPERIMENT_DIRECTORY, "plots")
    else:
        DATA_ROOT  =  os.path.join(mnt_point, "Lab Matteoli", "Silva", "projects", "data", EXPERIMENT_DIRECTORY)
        PLOTS_ROOT = os.path.join(mnt_point, "Lab Matteoli", "Silva", "projects", "results", EXPERIMENT_DIRECTORY, "plots")

data_input_path = os.path.join(DATA_ROOT, "QuPath_output")
data_output_path= os.path.join(DATA_ROOT, "BraiAn_output")

if not(os.path.exists(data_output_path)):
    os.makedirs(data_output_path, exist_ok=True)

## The Allen Brain Atlas

We start by importing the mouse Allen Brain Atlas, in which we find information about all brain regions (their parent region and children regions in the brain hierarchy, for example).

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = os.path.join(project_path, "data", "AllenMouseBrainOntology.json")
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, BRANCHES_TO_EXCLUDE, use_literature_reuniens=USE_LITERATURE_REUNIENS)
path_to_summary_structures = os.path.join(project_path, "data", "AllenSummaryStructures.csv")
AllenBrain.select_from_csv(path_to_summary_structures, include_nre_tot=USE_LITERATURE_REUNIENS)
selected_regions = AllenBrain.get_selected_regions()
print(f"You selected {len(selected_regions)} Summary Structure regions.")

#parent_region = AllenBrain.parent_region
#direct_subregions = AllenBrain.direct_subregions
#full_name = AllenBrain.full_name
#regions = AllenBrain.list_all_subregions("root", mode="depth")

We can also visualize the hierarchy of brain regions as a network (a tree). **Note that running the above cell may take a few minutes**.

In [ ]:
## Plot brain region hierarchy
## If you want to plot it, install PyDot (pydot)
if PLOT_ALLENBRAIN_HIERARCHY:
    fig = AllenBrain.plot_plotly_graph()
    fig.show()

Based on the graph above, you might want to specify the regions on which you want to do further PLS analysis:  
*Note: to see more information about the regions, hover over them with your mouse.*

- Specify a level. Analysis can only be done on one level (slice) in the brain region.

- To exclude brain regions that belong to a certain branch, add the *abbreviated* nodes at the beginning of the branches to the list above.  
Example:  
```branches_to_exclude = ["retina", "VS"]```  
means that **all the subregions that belong to the retina and the ventricular systems** are excluded from the PLS analysis.

## Load data

Now, we're ready to read the ```.csv``` files with the cell counts, and also the exclusion files (if there were regions to exclude).  
Below, you have to specify:
- ```animals_root```: Absolute path to the folder that contains the animal folders.
- ```group_1_dirs```: A list of names of the folders corresponding to animals in **Group 1** (e.g., Control group). Indeed, it is necessary to store the results in individual folders for each animal.
- ```group_2_dirs```: A list of names of the folders corresponding to animals in **Group 2** (e.g., Stress group).
- ```group_1_name```: A meaningful string for Group 1.
- ```group_2_name```: A meaningful string for Group 2.
- ```area_key```: A string of the column in the ```.csv``` files that refers to the size of a brain areatra
- ```tracer_key```: A string of the column in the ```.csv``` files that refers to the tracer number used to highlight the marker
- ```marker```: A string of the marker we would like to highlight (e.g. CFos)

Provare a modificar per ottenere densita in mm^2 (da micron)

Now, we load the Control and Stress results seperately in two pandas dataframes, and save the results.

**Note**: regions to exclude are automatically excluded.

In [ ]:
groups_slices: List[List[BraiAn.SlicedBrain]] = []
for i in range(len(groups)):
    group_slices = []
    for animal_dir in groups[i].dirs:
        if not os.path.isdir(os.path.join(data_input_path, animal_dir)):
            print(f"WARNING: could not find the directory '{animal_dir}' in '{EXPERIMENT_DIRECTORY}'. Skipping this animal.")
            continue
        sliced_brain = BraiAn.SlicedBrain(animal_dir,
                                            os.path.join(data_input_path, animal_dir),
                                            AllenBrain,
                                            AREA_KEY,
                                            TRACER_KEY,
                                            MARKER,
                                            area_units="µm2")
        group_slices.append(sliced_brain)
    groups_slices.append(group_slices)
    print(f"Imported all brain slices from {str(len(groups[i].dirs))} animals of {groups[i].name} group.")

In [ ]:
if PLOT_ANIMALS_ROOTS:
    root_plot = BraiAn.plot_region_density("root", *groups_slices, width=1000, height=500)
    root_plot.show()

In [ ]:
region_name = "root"
import numpy as np

summed_brains = []
group_lengths = [len(group) for group in groups_slices]
n_brains_before_group = np.cumsum(group_lengths)
groups_roots = []
for i, group_slices in enumerate(groups_slices):
    n_brains_before = n_brains_before_group[i-1] if i > 0 else np.int64(0)
    group_summed_brains = [BraiAn.AnimalBrain(sliced_brain, hemisphere_distinction=False, use_literature_reuniens=USE_LITERATURE_REUNIENS) for sliced_brain in group_slices]
    summed_brains.extend(group_summed_brains)
    group_roots = []
    for j, sliced_brain in enumerate(group_slices):
        n_brain = n_brains_before+j
        sliced_brain = BraiAn.merge_sliced_hemispheres(sliced_brain)
        for slice in sliced_brain.slices:
            try:
                density = slice.data.loc[region_name, slice.marker] / slice.data.loc[region_name, "area"]
                # density = slice.data.loc[region_name, slice.marker]
            except KeyError as e:
                print(f"WARNING: Could not find the '{region_name}' region for image '{slice.name}' of {slice.animal}")
                continue
            group_roots.append(density)
    group_roots_avg = np.array(group_roots).mean()
    groups_roots.append(group_roots_avg)
for name, avg in zip([group.name for group in groups], groups_roots):
    print(f"{name}: {avg}")

In [ ]:
# print("N regions above threshold:", sum([(brain.data > cv_threshold).sum() for brain in cvar_brains]))
# print("N regions below threshold:", sum([(brain.data <= cv_threshold).sum() for brain in cvar_brains]))
if PLOT_COEFFICIENT_OF_VARIATION:
    cvar_plot = BraiAn.plot_cv_above_threshold(AllenBrain, *groups_slices, cv_threshold=PLOT_COEFFICIENT_OF_VARIATION_THRESHOLD, width=1000, height=500)
    cvar_plot.show()

In [ ]:
r = "IG"
n_group = 2
n_animal = 3
sliced_brain = groups_slices[n_group-1][n_animal-1]
sliced_brain = BraiAn.merge_sliced_hemispheres(sliced_brain)
all_slices_df = sliced_brain.concat_slices()
slices_per_area = all_slices_df.groupby(all_slices_df.index).count().iloc[:,0]
print(f"""Summary for brain region '{r}' of {sliced_brain.name}:
    - N slices: {slices_per_area[r]}
    - Mean: {BraiAn.AnimalBrain(sliced_brain, mode="avg", hemisphere_distinction=False, use_literature_reuniens=USE_LITERATURE_REUNIENS).data[r]:.2f} {sliced_brain.marker}/mm²),
    - S.D.: {BraiAn.AnimalBrain(sliced_brain, mode="std", hemisphere_distinction=False, use_literature_reuniens=USE_LITERATURE_REUNIENS).data[r]:.2f} {sliced_brain.marker}/mm²,
    - Coefficient of Variation: {BraiAn.AnimalBrain(sliced_brain, mode="cvar", hemisphere_distinction=False, use_literature_reuniens=USE_LITERATURE_REUNIENS).data[r]}
""")

In [ ]:
# NOTE: brains are being written WITH Left/Right discrimination
# If you desire to save them without, call AnimalBrain with hemisphere_distinction=False

groups_sum_brains: List[List[BraiAn.AnimalBrain]] = [[BraiAn.AnimalBrain(sliced_brain, mode=ANIMAL_BRAIN_MODE, use_literature_reuniens=USE_LITERATURE_REUNIENS) for sliced_brain in sliced_brain_list] for sliced_brain_list in groups_slices]
if SAVE_ANIMALS:
    for i in range(len(groups)):
        for animal in groups_sum_brains[i]:
            animal.write_all_brains(data_output_path)

In [ ]:
animal_groups: List[BraiAn.AnimalGroup] = [BraiAn.AnimalGroup(groups[i].name, groups_sum_brains[i], AllenBrain) for i in range(len(groups))]

In [ ]:
if SAVE_GROUPS:
    for i in range(len(groups)):
        animal_groups[i].to_csv(data_output_path, f"cell_counts_{groups[i].name}.csv", overwrite=True)